In [ ]:
# -*- coding: utf-8 -*-
"""
author SparkByExamples.com
"""

In [ ]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType

In [ ]:
spark = SparkSession.builder.appName('SparkByExamples.com').getOrCreate()

In [ ]:
columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders")]

In [ ]:
df = spark.createDataFrame(data=data,schema=columns)

In [ ]:
df.show(truncate=False)

In [ ]:
def convertCase(str):
    resStr=""
    arr = str.split(" ")
    for x in arr:
       resStr= resStr + x[0:1].upper() + x[1:len(x)] + " "
    return resStr 


 Converting function to UDF 
<br>
convertUDF = udf(lambda z: convertCase(z))<br>
df.select(col("Seqno"), \<br>
    convertUDF(col("Name")).alias("Name") ) \<br>
.show(truncate=False)<br>
@udf(returnType=StringType()) <br>
def upperCase(str):<br>
    return str.upper()<br>
upperCaseUDF = udf(lambda z:upperCase(z),StringType())    <br>
df.withColumn("Cureated Name", upperCase(col("Name"))) \<br>
.show(truncate=False)<br>
Using UDF on SQL 


In [ ]:
spark.udf.register("convertUDF", convertCase,StringType())
df.createOrReplaceTempView("NAME_TABLE")
spark.sql("select Seqno, convertUDF(Name) as Name from NAME_TABLE") \
     .show(truncate=False)
     
spark.sql("select Seqno, convertUDF(Name) as Name from NAME_TABLE " + \
          "where Name is not null and convertUDF(Name) like '%John%'") \
     .show(truncate=False)  
     
""" null check """

In [ ]:
columns = ["Seqno","Name"]
data = [("1", "john jones"),
    ("2", "tracey smith"),
    ("3", "amy sanders"),
    ('4',None)]

In [ ]:
df2 = spark.createDataFrame(data=data,schema=columns)
df2.show(truncate=False)
df2.createOrReplaceTempView("NAME_TABLE2")
    
spark.udf.register("_nullsafeUDF", lambda str: convertCase(str) if not str is None else "" , StringType())

In [ ]:
spark.sql("select _nullsafeUDF(Name) from NAME_TABLE2") \
     .show(truncate=False)

In [ ]:
spark.sql("select Seqno, _nullsafeUDF(Name) as Name from NAME_TABLE2 " + \
          " where Name is not null and _nullsafeUDF(Name) like '%John%'") \
     .show(truncate=False)  